In [ ]:
import os
from timeit import default_timer as timer
import pandas as pd
from datetime import datetime, timedelta, date
from glob import glob
from pathlib import Path
import pandas as pd
import numpy as np

In [ ]:
path_to_data='/scratch/spf248/covid/data'
start_date='01/01/2020'
end_date='04/16/2020'
source='cuebiq'
country='ID'
admin_id='ADM4_PCODE'
city_id='UC_NM_MN'
demographics=[
'wealth_index',
'population_density',
]
q_days=0.5
min_pings=20
day=list(range(8,20))
night=list(range(8))+list(range(20,24))

# Compute time spent at home

In [ ]:
print('Import data')
start = timer()
data_dir = Path(os.path.join(path_to_data,source,'processed',country,'pings_home'))
pings_home = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in data_dir.glob('*.parquet')
)
print("Done in", round(timer()-start), "sec")  

# Data processing

In [ ]:
print('Process')
start = timer()
pings_home.drop(['latitude','longitude','accuracy','classification_type','ADM4_PCODE'],1,inplace=True)
pings_home.sort_values(by=['cuebiq_id','time'],inplace=True)
pings_home['hour']=pings_home.time.dt.hour
print("Done in", round(timer()-start), "sec")  

In [ ]:
print('Duration between users pings')
start = timer()
time_spent=pd.concat([pings_home.rename(columns={'time':'time_start','hour':'hour_start','is_home':'home_start'}),
pings_home.groupby('cuebiq_id').shift(-1).rename(columns={'time':'time_end','hour':'hour_end','is_home':'home_end'})],1)
time_spent.reset_index(drop=True,inplace=True)
time_spent.dropna(inplace=True)
time_spent['duration']=time_spent.time_end.subtract(time_spent.time_start)
time_spent['is_home']=((time_spent.home_end.add(time_spent.home_start)==2).astype(int))
print("Done in", round(timer()-start), "sec")  

In [ ]:
print('Compute time spent at home')
start = timer()
time_home=time_spent[(time_spent.is_home==1)&(time_spent.time_start>=pd.to_datetime(start_date))&(time_spent.time_start<=pd.to_datetime(end_date))].groupby(['cuebiq_id',pd.Grouper(key='time_start',freq='D')])['duration'].sum()
daytime_home=time_spent[(time_spent.is_home==1)&(time_spent.hour_start.isin(day))&(time_spent.hour_end.isin(day))&(time_spent.time_start>=pd.to_datetime(start_date))&(time_spent.time_start<=pd.to_datetime(end_date))].groupby(['cuebiq_id',pd.Grouper(key='time_start',freq='D')])['duration'].sum()
nighttime_home=time_spent[(time_spent.is_home==1)&(time_spent.hour_start.isin(night))&(time_spent.hour_end.isin(night))&(time_spent.time_start>=pd.to_datetime(start_date))&(time_spent.time_start<=pd.to_datetime(end_date))].groupby(['cuebiq_id',pd.Grouper(key='time_start',freq='D')])['duration'].sum()
time_home.rename(columns={'time_start':'date'},inplace=True)
daytime_home.rename(columns={'time_start':'date'},inplace=True)
nighttime_home.rename(columns={'time_start':'date'},inplace=True)
print("Done in", round(timer()-start), "sec")

In [ ]:
print('Save')
start = timer()
time_home.to_csv(os.path.join(path_to_data,source,'processed',country,'time_home_date_id.csv'))
daytime_home.to_csv(os.path.join(path_to_data,source,'processed',country,'daytime_home_date_id.csv'))
nighttime_home.to_csv(os.path.join(path_to_data,source,'processed',country,'nighttime_home_date_id.csv'))
print("Done in", round(timer()-start), "sec")

# Figures

In [ ]:
%matplotlib inline
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
print('Load Results')
start = timer()
time_home=pd.read_csv(os.path.join(path_to_data,source,'processed',country,'time_home_date_id.csv'),index_col=0)
print("Done in", round(timer()-start), "sec")

In [ ]:
print('Import')
start = timer()
data_dir = Path(os.path.join(path_to_data,source,'processed',country,'n_pings_id_date_hour'))
n_pings_id_date_hour = pd.concat(
    pd.read_parquet(parquet_file)
    for parquet_file in data_dir.glob('*.parquet')
)
print("Done in", round(timer()-start), "sec")  

In [ ]:
print('Process')
start = timer()
time_home.date=pd.to_datetime(time_home.date)
time_home.duration=pd.to_timedelta(time_home.duration)
print("Done in", round(timer()-start), "sec")

In [ ]:
n_pings_id_date_hour.groupby('cuebiq_id').agg(n_days=('date','nunique'))

In [ ]:
activity_id=time_home.groupby('cuebiq_id').agg(n_days=('date','count'),n_pings=('duration','count'))

In [ ]:
time_home.head()

In [ ]:
activity_id.head()

In [ ]:
time_home[time_home.cuebiq_id.isin(activity_id[activity_id['n_days']>=activity_id['n_days'].quantile(0.0)].index)
].groupby('date')['duration'].apply(lambda x:(x/np.timedelta64(1, 'h')).median()).plot()

In [ ]:
(time_home / np.timedelta64(1, 'h')).describe()

In [ ]:
time_spent.head()

In [ ]:
time_spent.is_home.describe()

In [ ]:
time_spent.loc[time_spent.is_home,'duration'].describe()